### Environment Setup

In [1]:
import datetime

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import MobileNetV2

%load_ext tensorboard

### Ingest, Image Preprocessing and Augmentation

In [2]:
train_data_dir = "../../Google Drive/My Drive/498/Project/mushie_image_data/"
num_classes = 2
img_width, img_height = 224, 224
classes = ['poisonous', 'edible']
batch_size = 32

# NOTE: our model will have a single output node
# This means that an output of '0' means a prediction of poisonous,
# And an output of '1' means a prediction of edible
# To flip this, change the order of the classes above

In [3]:
# Define image augmentation methods here
# As well as the train/validation split (thanks Keras for adding that feature!)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

print("Training set:")
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    classes = classes,
    shuffle=True,
    class_mode='binary',
    subset='training')

print("Validation set:")
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    classes = classes,
    class_mode='binary',
    subset='validation')

Training set:
Found 7135 images belonging to 2 classes.
Validation set:
Found 1782 images belonging to 2 classes.


In [ ]:
### Modeling Setup

In [8]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))

In [9]:
print("These layers will be set to untrainable: ")
for layer in base_model.layers[:-11]:
    layer.trainable = False
    print(layer.name)

These layers will be set to untrainable: 
input_2
Conv1
bn_Conv1
Conv1_relu
expanded_conv_depthwise
expanded_conv_depthwise_BN
expanded_conv_depthwise_relu
expanded_conv_project
expanded_conv_project_BN
block_1_expand
block_1_expand_BN
block_1_expand_relu
block_1_pad
block_1_depthwise
block_1_depthwise_BN
block_1_depthwise_relu
block_1_project
block_1_project_BN
block_2_expand
block_2_expand_BN
block_2_expand_relu
block_2_depthwise
block_2_depthwise_BN
block_2_depthwise_relu
block_2_project
block_2_project_BN
block_2_add
block_3_expand
block_3_expand_BN
block_3_expand_relu
block_3_pad
block_3_depthwise
block_3_depthwise_BN
block_3_depthwise_relu
block_3_project
block_3_project_BN
block_4_expand
block_4_expand_BN
block_4_expand_relu
block_4_depthwise
block_4_depthwise_BN
block_4_depthwise_relu
block_4_project
block_4_project_BN
block_4_add
block_5_expand
block_5_expand_BN
block_5_expand_relu
block_5_depthwise
block_5_depthwise_BN
block_5_depthwise_relu
block_5_project
block_5_project_BN

In [10]:
# Now we set the highest convolution block to trainable
# But make sure to not allow any BatchNorm layers be trainable
# https://keras.io/guides/transfer_learning/#finetuning
for layer in base_model.layers[-11:]:
    if "BN" not in layer.name and "bn" not in layer.name:
        layer.trainable = True
        print("\033[93m Trainable: ", layer.name, "\033[0m")
    else:
        layer.trainable = False
        print("Untrainable: ", layer.name)

 Trainable:  block_16_expand 
Untrainable:  block_16_expand_BN
 Trainable:  block_16_expand_relu 
 Trainable:  block_16_depthwise 
Untrainable:  block_16_depthwise_BN
 Trainable:  block_16_depthwise_relu 
 Trainable:  block_16_project 
Untrainable:  block_16_project_BN
 Trainable:  Conv_1 
Untrainable:  Conv_1_bn
 Trainable:  out_relu 


In [11]:
# Add in new top layers
# With a sigmoid output node (so we can do binary classification)
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

### Tensorboard Evaluation

In [12]:
# To launch tensorboard, run this cell
# Enable auto-reloading in the settings menu (it looks like a gear)
!rm -rf ./logs/ 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

%tensorboard --logdir logs

### Training

In [13]:
save_checkpoints = tf.keras.callbacks.ModelCheckpoint(
    filepath='./tmp/checkpoint',
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

In [ ]:
#enable early stopping
es = tf.keras.callbacks.EarlyStopping(
                                 monitor='val_acc',
                                 patience=5,
                                 mode='auto',
                                 baseline=None,
                                 restore_best_weights=True
                                )

In [14]:
#compile and train the model
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adagrad(),
              metrics=['acc', 'AUC'])

In [ ]:
EPOCHS = 30
model.fit(train_generator,
          epochs = EPOCHS,
          steps_per_epoch = train_generator.samples // batch_size,
          validation_data=validation_generator,
          validation_steps = validation_generator.samples // batch_size,
          callbacks=[tensorboard_cb, save_checkpoints]
         )

Epoch 1/30
 68/222 [========>.....................] - ETA: 2:25 - loss: 0.7064 - acc: 0.5558 - auc: 0.5434

/Users/christophrico/.local/share/virtualenvs/mushie-classifier-hjQZkQ93/lib/python3.8/site-packages/PIL/Image.py:962: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


222/222 [==============================] - 283s 1s/step - loss: 0.6766 - acc: 0.5886 - auc: 0.5944 - val_loss: 0.6266 - val_acc: 0.6347 - val_auc: 0.6984
Epoch 2/30
222/222 [==============================] - 288s 1s/step - loss: 0.6023 - acc: 0.6787 - auc: 0.7211 - val_loss: 0.6091 - val_acc: 0.6591 - val_auc: 0.7245
Epoch 3/30
222/222 [==============================] - 277s 1s/step - loss: 0.5781 - acc: 0.6972 - auc: 0.7499 - val_loss: 0.6011 - val_acc: 0.6687 - val_auc: 0.7338
Epoch 4/30
222/222 [==============================] - 263s 1s/step - loss: 0.5698 - acc: 0.7035 - auc: 0.7622 - val_loss: 0.6162 - val_acc: 0.6602 - val_auc: 0.7199
Epoch 5/30
222/222 [==============================] - 263s 1s/step - loss: 0.5607 - acc: 0.7067 - auc: 0.7706 - val_loss: 0.6583 - val_acc: 0.6261 - val_auc: 0.7264
Epoch 6/30
222/222 [==============================] - 252s 1s/step - loss: 0.5304 - acc: 0.7214 - auc: 0.8032 - val_loss: 0.6025 - val_acc: 0.6693 - val_auc: 0.7305
Epoch 7/30
222/222 [=

In [12]:
model.save('saved_model/mushie_model_test.h5')

### Fine-Tuning

In [ ]:
# Unfreeze all the layers to do some fine-tuning
for layer in model.layers:
    # But make sure to not allow any BatchNorm layers be trainable
    # https://keras.io/guides/transfer_learning/#finetuning
    if "BN" not in layer.name and "bn" not in layer.name:
        layer.trainable = True
        print("Trainable: ", layer.name)
    else:
        layer.trainable = False
        print("\033[93m Untrainable: ", layer.name, "\033[0m")

In [ ]:
model.summary()

In [ ]:
# Compile the model to learn at an extremely slow rate
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adagrad(1e-5),
              metrics=['acc', 'AUC'])

In [ ]:
EPOCHS = 30
model.fit(train_generator,
          epochs = EPOCHS,
          steps_per_epoch = train_generator.samples // batch_size,
          validation_data=validation_generator,
          validation_steps = validation_generator.samples // batch_size
         )

In [ ]:
model.save('saved_model/mushie_model_partially_trained.h5')